In [ ]:
import pandas as pd
import pyodbc
df=pd.read_csv("clean_data_github.csv")
df_con=pd.read_csv("updated_data.csv")


# inserting data to Language table and TypeOwner Table

In [ ]:
import pyodbc
# Language
connection_string = 'Driver={SQL Server};Server=LAPTOP-5VUQJT8J\SQLEXPRESS;Database=GITHUB_DATA;UID=;PWD=;'
connection = pyodbc.connect(connection_string)
cursor = connection.cursor()

df_language_unique = df.drop_duplicates(subset=['language'])
for _, row in df_language_unique.iterrows():
    query = "INSERT INTO Language (language) VALUES (?)"
    values = (row['language'],)
    cursor.execute(query, values)

# TypeOwner
df_type_owner_unique = df.drop_duplicates(subset=['type_owner'])
for _, row in df_type_owner_unique.iterrows():
    query = "INSERT INTO TypeOwner (typeOwner) VALUES (?)"
    values = (row['type_owner'],)
    cursor.execute(query, values)


connection.commit()

cursor.close()
connection.close()



# inserting data to Repository table

In [ ]:


connection = pyodbc.connect(connection_string)
cursor = connection.cursor()

def get_id_type_owner(type_owner):
    cursor.execute("SELECT idTypeOwner FROM TypeOwner WHERE typeOwner = ?", type_owner)
    result = cursor.fetchone()
    return result[0] if result else None

def get_id_language(language):
    cursor.execute("SELECT idLanguage FROM Language WHERE language = ?", language)
    result = cursor.fetchone()
    return result[0] if result else None

# Iterate through the DataFrame rows and insert data into the "Repository" table
for _, row in df.iterrows():
    # Get idTypeOwner and idLanguage from the corresponding tables
    id_type_owner = get_id_type_owner(row['type_owner'])
    id_language = get_id_language(row['language'])
    
    # Check if the repository already exists in the table
    cursor.execute("SELECT COUNT(*) FROM Repository WHERE idRepo = ?", row['id'])
    if cursor.fetchone()[0] == 0:
        # Convert date strings to datetime objects
        insert_query = '''
                        INSERT INTO Repository (
                            idTypeOwner, idLanguage, fullName, createdAt, startCount, description, htmlUrl, contributorsUrl,
                            openIssuesCount_, forks
                        )
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'''
        values = (id_type_owner, id_language, row['full_name'], row['created_at'], row['stargazers_count'],
              row['description'], row['html_url'], row['contributors_url'], row['open_issues_count'], row['forks'])
        
        cursor.execute(insert_query,values)
connection.commit()
cursor.close()
connection.close()


# inserting data to Topic table

In [ ]:
import ast
df['topics'] = df['topics'].apply(ast.literal_eval)
df_topics=df.explode('topics')
df_topics_unique = df_topics.drop_duplicates(subset=['topics'])
df_topics_unique

connection = pyodbc.connect(connection_string)
cursor = connection.cursor()
df_topics=df.explode('topics')
df_topics_unique = df_topics.drop_duplicates(subset=['topics'])
for _, row in df_topics_unique.iterrows():
    query = "INSERT INTO Topic (topic) VALUES (?)"
    values = (row['topics'],)
    cursor.execute(query, values)
connection.commit()

cursor.close()
connection.close()

# inserting data to Repo_Topic table

In [ ]:
connection = pyodbc.connect(connection_string)
cursor = connection.cursor()

def get_id_topic(topic):
    cursor.execute("SELECT idTopic FROM Topic WHERE topic = ?", topic)
    result = cursor.fetchone()
    return result[0] if result else None

def get_id_repo(fullName):
    cursor.execute("SELECT idRepo FROM Repository WHERE fullName = ?", fullName)
    result = cursor.fetchone()
    return result[0] if result else None

try:
    connection.autocommit = False

    for _, row in df_topics.iterrows():
        id_topic = get_id_topic(row['topics'])
        id_repo = get_id_repo(row['full_name'])

        if id_topic is not None and id_repo is not None:
            query = "INSERT INTO Repo_Topic (idTopic, idRepo) VALUES (?, ?)"
            values = (id_topic, id_repo)

            cursor.execute(query, values)

    connection.commit()
except Exception as e:
    connection.rollback()
    print("Error occurred during data insertion:", str(e))

finally:
    cursor.close()
    connection.close()


# inserting data to Contributor table

In [ ]:
connection = pyodbc.connect(connection_string)
cursor = connection.cursor()
df_sql_cont_df_unique = new_df_con.drop_duplicates(subset=['login'])
for _, row in df_sql_cont_df_unique.iterrows():
    query = "INSERT INTO Contributor (contrubtor) VALUES (?)"
    values = (row['login'],)
    cursor.execute(query, values)

connection.commit()
cursor.close()
connection.close()

# inserting data to Contribution table

In [ ]:
connection = pyodbc.connect(connection_string)
cursor = connection.cursor()

def get_id_Contributor(Contributor):
    cursor.execute("SELECT idContrubutor FROM Contributor WHERE contrubtor = ?", Contributor)
    result = cursor.fetchone()
    return result[0] if result else None

def get_id(id_fl):
    cursor.execute("SELECT idRepo FROM Repository WHERE fullName = ?", id_fl)
    result = cursor.fetchone()
    return result[0] if result else None

try:
    connection.autocommit = False

    for _, row in df_sql_cont_df_unique.iterrows():
        # Get idContrubutor from the 'Contributor' table
        id_contrubutor = get_id_Contributor(row['login'])
        id_fl = get_id(row['full_name'])

        # Check if 'idRepo' is not None, meaning it was found in the 'Repository' table
        if id_fl is not None:
            query = "INSERT INTO Contribution (idContrubutor, idRepo, number_Contributions) VALUES (?, ?, ?)"
            values = (
                id_contrubutor,
                id_fl,
                row['contributions'],
            )
            cursor.execute(query, values)

    connection.commit()
except Exception as e:
    connection.rollback()
    print("Error occurred during data insertion:", str(e))

finally:
    cursor.close()
    connection.close()
